## The Problem: Imagine that you manage the design and technology for a collection of 256 advertising placements with a single publisher with websites devoted to golf; in this context a publisher is simply a website owner. Each placement is called an ad-unit and is associated with the ad-unit id= 1::256. The term ’ad-call’ is used to label the event when your server is asked for the data required to render the ad-unit for the user. Each ad-unit can be rendered in one of two treatments, called treatment 0 or 1. Perhaps treatments 0 and 1 differ in the size of the advertiser logos. Your technology group has afforded you a knob for each ad-unit, in the form of a number p[0:1]. Upon an ad-call for ad-uniti, your adserver draws a random number, and if this number is less than p then treatment 1 is shown, else treatment 0 is shown. To simplify this problem, assume that each adunit has exactly 1,000 ad-calls per day, and that each of the 256,000 ad-calls each day is due to a distinct user. Upon the ad-call, the user can interact with the ad-unit for an arbitrary number of seconds. For each second of interaction, an advertiser pays the publisher a cps (cost-per-second) c_ai which is a value specific to each ad-unit id_i and treatment id_a. You have an agreement with the publisher, that each day, the total advertiser spend divided by the total number of seconds of interaction, is no less than 0.20. Additionally, you have an internal requirement, that each day, the total number of adcalls (summed over all ad-units) with treatment 0 is equal to the number of adcalls with treatment 1.Your task is to choose the values of p_1::p_256 to maximize the expected total number of seconds of interaction (sum over all adcalls) that occur tomorrow, while guaranteeing both the publisher agreement and the internal requirement (in expectation).

In [1]:
import numpy 
import matplotlib 
from matplotlib import pyplot
import seaborn as sns 
import pandas
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
ad_info = pandas.read_json("data/ad_calls.json")

In [3]:
ad_info.keys()

Index([u'ad_unit_id', u'cost_per_second', u'n_seconds', u'treatment_id'], dtype='object')

#### Casually glancing through the file I see that treatment 1 always costs 0.32 c/s and 0 costs 0.16 c/s. Reformat the table so I can get the average values for treatment 0 and 1 for each of the ad_unit_ids (pandas pivot table automatically aggregates the entries by the mean for the table value)

In [4]:
ad_mean_pivot = pandas.pivot_table(ad_info,values='n_seconds', index=['ad_unit_id'],columns=['treatment_id'])

In [5]:
test = ad_info['n_seconds'][(ad_info['ad_unit_id']==0)&(ad_info['treatment_id']==1)]
print numpy.mean(test)

7.63993288107


In [6]:
ad_mean_pivot.head(n=5)

treatment_id,0,1
ad_unit_id,,
0,2.173420,7.639933
1,1.857541,3.647597
2,1.935385,3.941385
3,1.960112,8.335809
4,1.942295,3.814124


In [7]:
zs = ad_mean_pivot[0].copy()
zs.sort(ascending=False)
os = ad_mean_pivot[1].copy()
os.sort(ascending=False)

In [8]:
zs.head(n=200)

ad_unit_id
198           2.275911
6             2.256219
31            2.253210
137           2.211310
8             2.205088
19            2.194322
116           2.188543
23            2.187440
0             2.173420
50            2.172567
138           2.171278
196           2.162835
108           2.162740
179           2.154072
152           2.148492
...
136           1.946941
61            1.946785
39            1.945064
96            1.944691
93            1.943890
4             1.942295
147           1.939751
150           1.939092
193           1.938938
139           1.937789
190           1.937271
64            1.937037
219           1.936628
84            1.935875
2             1.935385
Name: 0, Length: 200, dtype: float64

In [9]:
os.head(n=200)

ad_unit_id
211           8.963853
60            8.866322
22            8.808542
201           8.784608
11            8.706966
168           8.684148
139           8.651411
240           8.595362
98            8.535774
52            8.495709
229           8.484473
148           8.450355
184           8.435209
192           8.427792
29            8.423214
...
232           3.997956
125           3.995327
187           3.993126
210           3.992027
132           3.990839
73            3.990039
8             3.989529
254           3.981833
58            3.977036
69            3.972680
10            3.968735
56            3.961485
158           3.958636
18            3.956286
84            3.955478
Name: 1, Length: 200, dtype: float64

#### By eye the greedy algorithm solution to this problem seems simple to solve by eye, because mode 1 pays more (0.32 compared to 0.16) and also makes users interact with the ads longer. Without doing any coding at all I expect the solution to be 1)For the first half of ads, sorted by the mode 1 durations, always choose mode 1 2)Once you have used up all your mode 1 trials for the day start choosing mode 0. 

### To solve the generic problem (where perhaps I do not know that the time AND price is always greater for mode 1 etc.), I will use a greedy algorithm strategy. In the first pass through the list I will greedily choose the option that optimizes the time until I run out of uses of that mode (each mode gets 256,000/2 uses only based on the constraints). On the second pass through the algorithm I will force the algorithm to switch the mode to the mode that makes the most money*time until the total price/duration > 0.2 (the second constraint) and the number of mode 1 uses equals the number of mode 0 uses. The final solution will give the optimal weights for mode 0 and 1 for all the ads.

In [10]:
weight_dict = {key: [0,0] for key in xrange(0,256)}
cur_m1 = 128000
cur_m0 = 128000

In [11]:
while(len(os)>0):
    cur_uses = 1000
    m1_key = os.keys()[0]
    m0_key = zs.keys()[0]
    m1_dur = os[m1_key]
    m0_dur = zs[m0_key]
    #print len(os),len(zs),m1_key,m0_key,m1_dur,m0_dur
    while(cur_uses>0):
        if(m1_dur>=m0_dur):
            #print 'M1'
            if(cur_m1>0):
                weight_dict[m1_key][1]+=1
                cur_uses-=1
                cur_m1-=1
            else:
                weight_dict[m1_key][0]+=1
                cur_uses-=1
                cur_m0-=1
            if(cur_uses==0):
                os.pop(m1_key)
                zs.pop(m1_key)
        else:
            #print 'M0'
            if(cur_m0>0):
                weight_dict[m0_key][0]+=1
                cur_uses-=1
                cur_m0-=1
            else:
                weight_dict[m0_key][1][0]+=1
                cur_uses-=1
                cur_m0-=1
            if(cur_uses==0):
                os.pop(m0_key)
                zs.pop(m0_key)
    

In [12]:
#Test to check that total sum of uses for mode 0 and mode 1 are both 128000
